<h1 style="color:rgb(0,120,170)">344.038, KV Multimedia Search and Retrieval (WS2023/24)</h1>
<h2 style="color:rgb(0,120,170)">Task 2_Group B</h2>

| First Name | Family Name  | Matr.Nr   |
|:-----------|:-------------|:----------|
| Harald     | Eibensteiner | K01300179 |
| Hadi       | Sanaei       | K11733444 |
| Lukas      | Troyer       | K12006666 |
| Lukas      | Wagner       | K01357626 |
| Branko     | Paunović     | K12046370 |

### Load Data & Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
information_data = pd.read_csv('datasets/id_information_mmsr.tsv', delimiter='\t')
mfcc_bow_data = pd.read_csv('datasets/id_mfcc_bow_mmsr.tsv', delimiter='\t')
mfcc_stats_data = pd.read_csv('datasets/id_mfcc_stats_mmsr.tsv', delimiter='\t')

### Audio-based(similarity, MFCC)

In [3]:
# Function to calculate similarity between two tracks
def calculate_similarity(query_features, target_features):
    similarity_matrix = cosine_similarity(query_features, target_features)
    return similarity_matrix

# Function to retrieve top N similar tracks
def retrieve_top_similar_tracks(query_track_id, features_data, feature_columns, information_data, num_top_similar):
    if query_track_id not in features_data['id'].values:
        print(f"Track ID {query_track_id} not found in the data.")
        return
    
    query_track_features = features_data[features_data['id'] == query_track_id].iloc[:, 1:].values

    # Calculate similarity with the given features
    similarity_matrix = calculate_similarity(query_track_features, features_data.iloc[:, 1:].values)

    # Get the indices of the top N similar tracks
    top_indices = np.argsort(similarity_matrix[0])[-num_top_similar:][::-1]

    # Create a DataFrame to store the results
    result_df = pd.DataFrame(columns=['Track ID', 'Similarity', 'Artist', 'Song'])


    # Create a list to store rows
    result_rows = []

    # Populate the list with song and artist information
    for track_index in top_indices:
        track_id = features_data.loc[track_index, 'id']

        info_row = information_data[information_data['id'] == track_id]
        if not info_row.empty:
            artist = info_row['artist'].values[0]
            song = info_row['song'].values[0]

            result_rows.append({
                'Track ID': track_id,
                'Similarity': similarity_matrix[0][track_index],
                'Artist': artist,
                'Song': song
            })
            
    # Create the DataFrame from the list of dictionaries
    result_df = pd.DataFrame(result_rows)

    # Sort the DataFrame based on "Similarity" column
    result_df = result_df.sort_values(by='Similarity', ascending=False)

    # Display sorted results
    print(f"\nTop {num_top_similar} Similar Tracks:")
    print(result_df)


def main():

    num_top_similar = int(input("Enter the number of top similar tracks: "))
    retrieve_top_similar_tracks("GDGURAgKxNPEuXzM", mfcc_bow_data, mfcc_bow_data.columns[1:], information_data, num_top_similar)
    retrieve_top_similar_tracks("GDGURAgKxNPEuXzM", mfcc_stats_data, mfcc_stats_data.columns[1:], information_data, num_top_similar)

if __name__ == "__main__":
    main()

    
# Sample query ID:  GDGURAgKxNPEuXzM

Enter the number of top similar tracks: 3

Top 3 Similar Tracks:
           Track ID  Similarity          Artist  \
0  GDGURAgKxNPEuXzM    1.000000  Monster Magnet   
1  1fmfftKuLUstbtui    0.940009     Iron Maiden   
2  W0I0QY8ucb4NcE3y    0.931293         Placebo   

                               Song  
0  Look to Your Orb for the Warning  
1          When the Wild Wind Blows  
2                       Haemoglobin  

Top 3 Similar Tracks:
           Track ID  Similarity                 Artist  \
0  GDGURAgKxNPEuXzM    1.000000         Monster Magnet   
1  c0oTPqvKZKkIyvkx    0.907322               Ladytron   
2  48mN3L6l1FWVFNUX    0.906402  The Birthday Massacre   

                               Song  
0  Look to Your Orb for the Warning  
1                         Deep Blue  
2                         Goodnight  


### Audio-based(similarity, BLFs)

### Audio-based(similarity, i-vectors)

### Audio-based(similarity, musicnn)

### Evaluation